# Tweet Variations and Examples

In [1]:
import psycopg2
import pandas as pd
import config
from matplotlib import pylab as plt
import seaborn as sns
import numpy as np
from IPython.display import display, HTML
import re
from IPython.display import HTML, display


color_palette = sns.color_palette(palette='muted', n_colors=None, desat=.75)
sns.set(context='notebook', palette=color_palette, style='whitegrid', font='sans-serif', font_scale=1.5, color_codes=False, rc=None)
pd.set_option('display.max_colwidth', -1)
table_styles = [{'selector': 'td',
                 'props': [('min-width', '100px'), ('text-align', 'center')]},
                {'selector': 'tr',
                 'props': [('border-bottom', '1px dotted black')]},
                {'selector': 'th',
                 'props': [('text-align', 'center')]}
               ]

%matplotlib inline

directory = "url_top_lists/"
stream = "comparison"

In [2]:
conn = None
try:
    # read connection parameters
    paramsS17 = config.cfgAzureS17()
    paramsS03 = config.cfgAzureS03()

    paramsF17 = config.cfgAzureF17()
    paramsF03 = config.cfgAzureF03()
    
    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    connS17 = psycopg2.connect(**paramsS17)
    connS03 = psycopg2.connect(**paramsS03)
    
    connF17 = psycopg2.connect(**paramsF17)
    connF03 = psycopg2.connect(**paramsF03)

    # create a cursor
    curS17 = connS17.cursor()
    curS03 = connS03.cursor()
    
    curF17 = connF17.cursor()
    curF03 = connF03.cursor()

    # execute a statement
    print('PostgreSQL database version:')
    
    curS17.execute('SELECT version()')
    curS03.execute('SELECT version()')
    curF17.execute('SELECT version()')
    curF03.execute('SELECT version()')
    
    # display the PostgreSQL database server version
    db_version_curS17 = curS17.fetchone()
    db_version_curS03 = curS03.fetchone()
    db_version_curF17 = curF17.fetchone()
    db_version_curF03 = curF03.fetchone()
    
    print(db_version_curS17)
    print(db_version_curS03)
    print(db_version_curF17)
    print(db_version_curF03)

    # close the communication with the PostgreSQL
    curS17.close()
    curS03.close()
    curF17.close()
    curF03.close()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 9.6.7, compiled by Visual C++ build 1800, 64-bit',)
('PostgreSQL 9.6.7, compiled by Visual C++ build 1800, 64-bit',)
('PostgreSQL 9.6.7, compiled by Visual C++ build 1800, 64-bit',)
('PostgreSQL 9.6.7, compiled by Visual C++ build 1800, 64-bit',)


In [3]:
tweetsS03 = pd.read_sql_query('''SELECT id, ti.text, is_retweet, retweet_id, is_quote, quote_id, is_reply, reply_to_status_id, tc.has_icard, tc.response_code FROM tweets_info as ti 
                                 INNER JOIN tweets_icards as tc ON ti.id = tc.tweet_id;''', connS17 )

print("Number of Tweets: %s" %len(tweetsS03))
tweetsS03.head()

Number of Tweets: 112002


,id,text,is_retweet,retweet_id,is_quote,quote_id,is_reply,reply_to_status_id,has_icard,response_code
0,988175619098861568,Beim nächsten Ton ist es 0 Uhr: ♩♩♩,False,NaN,False,NaN,False,NaN,False,NaN
1,988175627453952002,[d] DiplIngMarkus Verbrauch gestern: 3.71 kWh Uptime: 41 days 21:01:04 since 23:48:32 13/10/2017,False,NaN,False,NaN,False,NaN,False,NaN
2,988175778474020868,Am Rajenbaumturm ★ IRONMAN Challenge | Nachtelf Druide #25 | World of Warcraft | Aloexis: https://t.co/GG6sCorejS via @YouTube,False,NaN,False,NaN,False,NaN,True,200.0
3,988175828801540097,Ab jetzt ist für 24h Montag,False,NaN,False,NaN,False,NaN,False,NaN
4,987388203593322496,I’m awaiting the call. https://t.co/baLXYp8JYi,False,NaN,False,NaN,False,NaN,False,NaN


In [4]:
def printExamples(data_set, data):
    print('Example:')
    display(HTML(data_set[['id', 'text']].iloc[0].to_frame().to_html()))
    print("Try this link: http://twitter.com/statuses/%s\n" %(data_set['id'].iloc[0]))
    
    if data_set['is_retweet'].iloc[0]:
        print("\nTweet is based on this Retweet:")
        display(HTML(data[data['id'] == data_set['retweet_id'].iloc[0]].to_html()))
        
    if data_set['is_quote'].iloc[0]:
        print("\nTweet is based on this Quote:")
        display(HTML(data[data['id'] == data_set['quote_id'].iloc[0]].to_html()))
    
    if data_set['is_reply'].iloc[0]:
        print("\nTweet is a reply to this Tweet:")
        display(HTML(data[data['id'] == data_set['reply_to_status_id'].iloc[0]].to_html()))
        
    
def printTweetInfo(data, is_retweet, is_quote, is_reply, has_icard):
    print("\n\n------------------")
    if has_icard:
        data_set = data[(data['is_retweet'] == is_retweet) & (data['is_quote'] == is_quote) 
                      & (data['is_reply'] == is_reply) & (data['has_icard'] == has_icard) & (data['response_code'] != None)]
        print("# Retweet: %s, Quote: %s, Reply: %s, iCard: %s \n## Percentage: %0.2f%%\tCount: %s\n" %(is_retweet, is_quote, is_reply, has_icard, len(data_set)/len(data)*100, len(data_set)))
    else:
        data_set = data[(data['is_retweet'] == is_retweet) & (data['is_quote'] == is_quote) 
                      & (data['is_reply'] == is_reply) & (data['has_icard'] == has_icard)]
        print("# Retweet: %s, Quote: %s, Reply: %s, iCard: %s \n## Percentage: %0.2f%%\tCount: %s\n" %(is_retweet, is_quote, is_reply, has_icard, len(data_set)/len(data)*100, len(data_set)))
   
    if len(data_set) > 0:
        printExamples(data_set, data)
    
    print("------------------\n\n")
    
printTweetInfo(tweetsS03, False, False, False, False)

printTweetInfo(tweetsS03, False, False, False, True)


printTweetInfo(tweetsS03, False, False, True, False)

printTweetInfo(tweetsS03, False, False, True, True)


printTweetInfo(tweetsS03, False, True, False, False)

printTweetInfo(tweetsS03, False, True, False, True)


printTweetInfo(tweetsS03, False, True, True, False)

printTweetInfo(tweetsS03, False, True, True, True)


printTweetInfo(tweetsS03, True, False, False, False)

printTweetInfo(tweetsS03, True, False, False, True)


printTweetInfo(tweetsS03, True, False, True, False)

printTweetInfo(tweetsS03, True, False, True, True)


printTweetInfo(tweetsS03, True, True, False, False)

printTweetInfo(tweetsS03, True, True, False, True)


printTweetInfo(tweetsS03, True, True, True, False)

printTweetInfo(tweetsS03, True, True, True, True)



------------------
# Retweet: False, Quote: False, Reply: False, iCard: False 
## Percentage: 33.21%	Count: 37201

Example:


,0
id,988175619098861568
text,Beim nächsten Ton ist es 0 Uhr: ♩♩♩


Try this link: http://twitter.com/statuses/988175619098861568

------------------




------------------
# Retweet: False, Quote: False, Reply: False, iCard: True 
## Percentage: 13.49%	Count: 15107

Example:


,2
id,988175778474020868
text,Am Rajenbaumturm ★ IRONMAN Challenge | Nachtelf Druide #25 | World of Warcraft | Aloexis: https://t.co/GG6sCorejS via @YouTube


Try this link: http://twitter.com/statuses/988175778474020868

------------------




------------------
# Retweet: False, Quote: False, Reply: True, iCard: False 
## Percentage: 22.86%	Count: 25599

Example:


...

Try this link: http://twitter.com/statuses/988295978842316801


Tweet is based on this Retweet:


,id,text,is_retweet,retweet_id,is_quote,quote_id,is_reply,reply_to_status_id,has_icard,response_code
57140,987987749612478464,"Weil immer wieder die Frage auftaucht, wo man rassistische Inhalte melden kann:\n\nhttps://t.co/N9eLUmmdSC\n\nhttps://t.co/hS94ZjPvvR (auch auf Twitter)\n\nDanke @NinaHoraczek (der Artikel hat mich angeregt, die Links rauszusuchen) https://t.co/2WXX94PgnI",False,NaN,True,9.879461e+17,False,NaN,True,200.0



Tweet is based on this Quote:


,id,text,is_retweet,retweet_id,is_quote,quote_id,is_reply,reply_to_status_id,has_icard,response_code
1302,987946091235725312,"“Wir müssen sehr wachsam sein”: Die österreichische Journalistin und Gatterer-Preisträgerin @NinaHoraczek über Populismus und Dumpfheit, die Stimmung im schwarz-blauen Österreich und eine Jobgarantie für kritische JournalistInnen. https://t.co/tHKzIgYS4h",False,NaN,False,NaN,False,NaN,True,200.0


------------------




------------------
# Retweet: True, Quote: True, Reply: True, iCard: False 
## Percentage: 0.00%	Count: 0

------------------




------------------
# Retweet: True, Quote: True, Reply: True, iCard: True 
## Percentage: 0.00%	Count: 0

------------------


